In [55]:

%matplotlib inline
import matplotlib.pyplot as mp
import urllib2
import urllib
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import re

#variables
Basic_Stats = pd.read_csv("./nflstatistics/Basic_Stats.csv");
Basic_Stats = Basic_Stats['Current Team'].replace(np.nan,'Retired');
print type(Basic_Stats)
Basic_Stats = Basic_Stats.drop(columns=['Number'])
#Basic_Stats = Basic_Stats.dropna(axis = 0, how="any")
#print Basic_Stats['Current Team']
print Basic_Stats

  

AttributeError: 'DataFrame' object has no attribute 'to_frame'